In [ ]:
# default_exp utils

# utils
> Collection of utils.

In [ ]:
# hide
# from nbdev.showdoc import *

In [ ]:
#export
# hide
from fastai.basics import *
from fastai.callbacks import GeneralScheduler, TrainingPhase

# Parameters parser

In [ ]:
# export
def param_parser(func):
    "Look at params in func and return an dict of args: default"
    p ={}
    params = inspect.signature(func).parameters
    for k, v in params.items():
        if v.default != inspect.Parameter.empty: p[k] = v.default
    return p

In [ ]:
param_parser(optim.Adam)

{'lr': 0.001,
 'betas': (0.9, 0.999),
 'eps': 1e-08,
 'weight_decay': 0,
 'amsgrad': False}

# Lr finder and Plot

In [ ]:
# export
def lr_f(learn):
    learn.lr_find()
    learn.recorder.plot()
    learn.p.min_lr = learn.recorder.lrs[learn.recorder.losses.index(min(learn.recorder.losses))] # todo check if learn.p exist
    print(f'min lr: {learn.p.min_lr:0.4f}')
# Learner.lr_f = lr_f

In [ ]:
# export
def plot(learn, plot_all=False):
    learn.recorder.plot_metrics()
    learn.recorder.plot_losses()
    if plot_all:
        learn.recorder.plot()
        learn.recorder.plot_lr(show_moms=True)
Learner.plot = plot

# Train

## FOC

In [ ]:
# export
# Basic train 
# Train learn with fit_one_cycle from fastai
def train(learn, show_graph=False):
    '''Run fit_one_cycle with learner, params in learn.p'''
    cbs = ShowGraph(learn) if show_graph else None
    print(f"epochs: {learn.p.epochs}, lr: {learn.p.lr:0.4f}, opt - {learn.p.opt_name}")
    learn.fit_one_cycle(learn.p.epochs, max_lr=learn.p.lr, div_factor=learn.p.div_factor, 
                        moms=learn.p.moms, pct_start=learn.p.pct_start, final_div=learn.p.final_div, callbacks=cbs)
Learner.train=train    

## FC from fastai

In [ ]:
# export
# Train learn with fit_fc from fastai.
def train_fc(learn, show_graph=False):
    '''Run fit_fc with learn, paramas in learn.p'''
    cbs = ShowGraph(learn) if show_graph else None
    print(f"epochs: {learn.p.epochs}, lr: {learn.p.lr:0.4f}, opt - {learn.p.opt_name}")
    learn.fit_fc(learn.p.epochs, lr=learn.p.lr, start_pct=learn.p.fc_start_pct, 
                        moms=learn.p.moms, callbacks=cbs)
# Learner.train_fc=train_fc    

## FC with refactoring. Lr and MOM separated.

In [ ]:
# export
# Callback for Lr sheduller
def FCSchedLr(learn, tot_epochs:int=1,  lr:float=4e-3,  start_pct:float=0.72, 
            lr_final_div=None, curve='cosine'):
    """refactored version of FCFit from fastai, only Lr"""
    b_all = len(learn.data.train_dl) * tot_epochs
    b_flat = int(b_all * start_pct)
    b_reduce = b_all - b_flat
    curve_type=annealing_cos
#     if curve=="cosine":        curve_type=annealing_cos
#     elif curve=="linear":      curve_type=annealing_linear
#     elif curve=="exponential": curve_type=annealing_exp
#     else: raiseValueError(f"annealing type not supported {curve}")

    ph_lr_flat = TrainingPhase(b_flat).schedule_hp('lr', lr)
    lr = (lr, lr/lr_final_div) if lr_final_div else lr
    ph_lr_reduce = TrainingPhase(b_reduce).schedule_hp('lr', lr, anneal=curve_type)
    phases = [ph_lr_flat, ph_lr_reduce]
    return GeneralScheduler(learn, phases=phases)

In [ ]:
# MOM shcheduller
def FCSchedMom(learn, tot_epochs:int=1, moms:Floats=(0.95,0.999),
                          start_pct:float=0.72, shift=0.1, curve='cosine'):
    """refactored version of FCFit from fastai, only Lr"""
    shift_n = start_pct*shift
    b_all = len(learn.data.train_dl) * tot_epochs
    b_flat = int(b_all * (start_pct-shift_n))
    b_reduce = int(b_all*shift_n*2)
    b_finish = b_all - b_flat - b_reduce
        
    
    if curve=="cosine":        curve_type=annealing_cos
    elif curve=="linear":      curve_type=annealing_linear
    elif curve=="exponential": curve_type=annealing_exp
    else: raiseValueError(f"annealing type not supported {curve}")

    ph_flat = TrainingPhase(b_flat).schedule_hp('mom', moms[0])
    ph_reduce = TrainingPhase(b_reduce).schedule_hp('mom', (moms[0],moms[1]),anneal=curve_type)
    ph_finish = TrainingPhase(b_finish).schedule_hp('mom', moms[1])
    phases = [ph_flat, ph_reduce, ph_finish]
    return GeneralScheduler(learn, phases=phases)

In [ ]:
# export
# version with step
def FCSchedMom(learn, tot_epochs:int=1, moms:Floats=(0.95,0.999),
                          start_pct:float=0.72, shift=0.1, curve='cosine'):
    """refactored version of FCFit from fastai, only Lr"""
    shift_n = start_pct*shift
    b_all = len(learn.data.train_dl) * tot_epochs
    print('curve - ', curve)
    if curve=="cosine":
        curve_type=annealing_cos
        b_flat = int(b_all * (start_pct-shift_n))
        b_reduce = int(b_all*shift_n*2)
        b_finish = b_all - b_flat - b_reduce
        ph_flat = TrainingPhase(b_flat).schedule_hp('mom', moms[0])
        ph_reduce = TrainingPhase(b_reduce).schedule_hp('mom', (moms[0],moms[1]),anneal=curve_type)
        ph_finish = TrainingPhase(b_finish).schedule_hp('mom', moms[1])
        phases = [ph_flat, ph_reduce, ph_finish]
    
    elif curve=="step":
#         curve_type=annealing_linear
#     elif curve=="exponential": curve_type=annealing_exp
        b_flat = int(b_all * (start_pct-shift_n))
#         b_reduce = int(b_all*shift_n*2)
        b_finish = b_all - b_flat
        ph_flat = TrainingPhase(b_flat).schedule_hp('mom', moms[0])
#         ph_reduce = TrainingPhase(b_reduce).schedule_hp('mom', (moms[0],moms[1]),anneal=curve_type)
        ph_finish = TrainingPhase(b_finish).schedule_hp('mom', moms[1])
        phases = [ph_flat, ph_finish]
    
    
    
    else: raiseValueError(f"annealing type not supported {curve}")

    
    return GeneralScheduler(learn, phases=phases)

In [ ]:
# export
# train model with schedullers - saparate Lr and MOMs
def fc_lr_mom(learn:Learner, tot_epochs:int=1, lr:float=defaults.lr,  moms=False, start_pct:float=0.72,
                  wd:float=None, lr_final_div=None, shift=0.1, 
                  curve='cosine', callbacks:Optional[CallbackList]=None)->None:
    "Fit a model with Flat Cosine Annealing"
    max_lr = learn.lr_range(lr)
    callbacks = listify(callbacks)
    callbacks.append(FCSchedLr(learn, tot_epochs=tot_epochs,  lr=lr, start_pct=start_pct, lr_final_div=lr_final_div,curve=curve))
    if moms: callbacks.append(FCSchedMom(learn,  tot_epochs=tot_epochs, moms=moms, start_pct=start_pct,shift=shift,curve=curve))
    learn.fit(tot_epochs, max_lr, wd=wd, callbacks=callbacks)

In [ ]:
# export
# version for use with experiment. 
# TODO - refactor exp for automate it.
def fc(learn, show_graph=False):
    '''Run fc scheduled lr, mom, with learn, paramas in learn.p'''
    cbs = ShowGraph(learn) if show_graph else None
    print(f"epochs: {learn.p.epochs}, lr: {learn.p.lr:0.4f}, opt - {learn.p.opt_name}")
    fc_lr_mom(learn, learn.p.epochs, lr=learn.p.lr, 
                    start_pct=learn.p.fc_start_pct, lr_final_div=learn.p.final_div,
                        moms=learn.p.moms, shift=learn.p.shift, callbacks=cbs) # !wd

# end

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_experiment.ipynb.
Converted 01_utils.ipynb.
Converted 02_consistency.ipynb.
Converted 03_data_imagenette.ipynb.
Converted 50_ranger.ipynb.
Converted index.ipynb.
